In [88]:
pip install pyspark

In [89]:
import os
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [90]:
# Download the Postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-20 22:44:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-11-20 22:44:27 (6.63 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [91]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Capstone").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [92]:
from pyspark import SparkFiles
url = "https://ellenbrafferty-bucket.s3.amazonaws.com/capstone_data/Flights_Aug_2021.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+----+-----+------------+----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+---------------+-------------------+-------------------+----+--------------+--------------+------------+------------+-------------+------------+---------+--------+-------+--------+-------------+-------------+---------+--------------+-------------------+----+
|YEAR|MONTH|DAY_OF_MONTH|   FL_DATE|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|ORIGIN_STATE_NM|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME| DEST_STATE_NM|CRS_DEP_TIME|CRS_ARR_TIME|ARR_DELAY_NEW|ARR_TIME_BLK|CANCELLED|AIR_TIME|FLIGHTS|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|
+----+-----+------------+----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+--------------

In [93]:
from pyspark.sql.functions import col, unix_timestamp, to_date
df = df.withColumn('flight_date', 
                   to_date("FL_DATE", 'yyyy-MM-dd'))

In [94]:
df = df.drop('FL_DATE','_c30')

In [95]:
df.show()

+----+-----+------------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+---------------+-------------------+-------------------+----+--------------+--------------+------------+------------+-------------+------------+---------+--------+-------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|YEAR|MONTH|DAY_OF_MONTH|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|ORIGIN_STATE_NM|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME| DEST_STATE_NM|CRS_DEP_TIME|CRS_ARR_TIME|ARR_DELAY_NEW|ARR_TIME_BLK|CANCELLED|AIR_TIME|FLIGHTS|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|flight_date|
+----+-----+------------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+---------------+-----------------

In [96]:
ml_cols = ['YEAR', 'MONTH', 'DAY_OF_MONTH', 'OP_CARRIER_FL_NUM','ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_NM', 'DEST_AIRPORT_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_NM', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'ARR_DELAY_NEW', 'CANCELLED', 'AIR_TIME', 'FLIGHTS', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
ml_df = df.select(ml_cols).withColumnRenamed('OP_CARRIER_FL_NUM', 'FLIGHT_NUM').withColumnRenamed('CRS_DEP_TIME', 'DEPT_TIME').withColumnRenamed('CRS_ARR_TIME', 'ARR_TIME').withColumnRenamed('ARR_DELAY_NEW', 'ARR_DELAY').withColumnRenamed('ORIGIN', 'ORIGIN_AIRPORT_CODE').withColumnRenamed('DEST', 'DEST_AIRPORT_CODE')

In [97]:
ml_df.show()

+----+-----+------------+----------+-----------------+-------------------+----------------+---------------+---------------+-----------------+--------------+--------------+---------+--------+---------+---------+--------+-------+--------+-------------+-------------+---------+--------------+-------------------+
|YEAR|MONTH|DAY_OF_MONTH|FLIGHT_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_CODE|ORIGIN_CITY_NAME|ORIGIN_STATE_NM|DEST_AIRPORT_ID|DEST_AIRPORT_CODE|DEST_CITY_NAME| DEST_STATE_NM|DEPT_TIME|ARR_TIME|ARR_DELAY|CANCELLED|AIR_TIME|FLIGHTS|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----+-----+------------+----------+-----------------+-------------------+----------------+---------------+---------------+-----------------+--------------+--------------+---------+--------+---------+---------+--------+-------+--------+-------------+-------------+---------+--------------+-------------------+
|2021|    8|           5|      5574|            10599|                

In [15]:
mode = "append"
from getpass import getpass
password = getpass('Enter database password')
jdbc_url="jdbc:postgresql://capstone.c9x4gosspizq.us-east-2.rds.amazonaws.com:5432/Flight_delays"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [87]:
ml_df.write.jdbc(url=jdbc_url, table='flight_delays', mode=mode, properties=config)